<a href="https://colab.research.google.com/github/anmyvla-beep/python-ai-Mysttonen-Angelika/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о телесериалах

**Цель**: Научиться читать несколько CSV-файлов из репозитория GitHub в Google Colab и выполнять базовую проверку данных о телесериалах с помощью pandas.

**Данные:**
- `tvserialone.csv` — информация о телесериалах: годы выхода, награды и номинации
- `tvserialtwo.csv` — информация о телесериалах: годы выхода и тематические категории (без наград)

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем оба CSV-файла в отдельные pandas DataFrame
3. Очищаем и анализируем структуру данных (сериалы, годы, награды, темы)
4. Проверяем качество данных и готовим их к дальнейшему анализу и возможному объединению

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🐱 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Mysttonen-Angelika"):
    !git clone -q https://github.com/anmyvla-beep/python-ai-Mysttonen-Angelika.git

%cd python-ai-Mysttonen-Angelika

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Mysttonen-Angelika")

/content/python-ai-Mysttonen-Angelika
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Mysttonen-Angelika


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [2]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd
import os

# Проверяем существование файлов
files = {
    "tvserialone.csv": "data/tvserialone.csv",
    "tvserialtwo.csv": "data/tvserialtwo.csv"
}

for name, path in files.items():
    if not os.path.exists(path):
        print(f"⚠️  Файл {name} не найден по пути: {path}")
    else:
        print(f"✅ Файл {name} найден")

print("\n" + "="*70)

# Читаем первый файл: сериалы с наградами
df_tv_one = pd.read_csv("data/tvserialone.csv")
print(f"✅ Загружено строк в df_tv_one (сериалы с наградами): {len(df_tv_one)}")
print(f"   Столбцы: {', '.join(df_tv_one.columns)}")

# Читаем второй файл: сериалы с тематикой
df_tv_two = pd.read_csv("data/tvserialtwo.csv")
print(f"\n✅ Загружено строк в df_tv_two (сериалы с тематикой): {len(df_tv_two)}")
print(f"   Столбцы: {', '.join(df_tv_two.columns)}")

print("\n" + "="*70)
print("💡 Оба датасета готовы к очистке и анализу")

✅ Файл tvserialone.csv найден
✅ Файл tvserialtwo.csv найден

✅ Загружено строк в df_tv_one (сериалы с наградами): 1224
   Столбцы: tv_series, tv_seriesLabel, start_year, end_year, awardLabel, subjectLabel

✅ Загружено строк в df_tv_two (сериалы с тематикой): 2679
   Столбцы: tv_series, tv_seriesLabel, start_year, end_year, subjectLabel

💡 Оба датасета готовы к очистке и анализу


## 🧹 [2B] Очистка и переименование столбцов для двух датасетов

В исходных CSV-файлах есть **технические столбцы** из Викиданных, которые мешают простому анализу:

- Столбец `tv_series` содержит URL Wikidata — нам достаточно читаемого названия из `tv_seriesLabel`
- Столбцы с суффиксом `Label` (`tv_seriesLabel`, `awardLabel`, `subjectLabel`) содержат человекочитаемые значения

**Структура файлов:**
| Файл | Особенности |
|------|-------------|
| `tvserialone.csv` | Есть награды: столбцы `awardLabel` + `subjectLabel` |
| `tvserialtwo.csv` | Только тематика: столбец `subjectLabel` (без наград) |

**В этом шаге мы для каждого датасета:**
- удалим столбец `tv_series` с URL Wikidata;
- переименуем `tv_seriesLabel → tv_series`, `awardLabel → award` (если есть), `subjectLabel → subject`;
- приведём числовые столбцы (`start_year`, `end_year`) к типу `int`.

> ⚠️ **Важно:** код написан идемпотентно — его можно запускать многократно без ошибок. Перед обработкой проверяется текущая структура столбцов.

In [3]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (идемпотентная версия для двух датасетов)

def clean_tv_dataset(df, name):
    """
    Идемпотентная очистка датасета с телесериалами.
    Возвращает очищенный датафрейм, не меняя исходный при повторных вызовах.
    """
    # Работаем с копией, чтобы не портить исходный датафрейм при повторных запусках
    df_clean = df.copy()

    # Проверяем, нужно ли выполнять очистку: если есть столбец с суффиксом "Label" — данные ещё не обработаны
    if "tv_seriesLabel" in df_clean.columns:
        # 1. Удаляем технический столбец с URL Wikidata (игнорируем ошибку, если столбца уже нет)
        df_clean = df_clean.drop(columns=["tv_series"], errors="ignore")

        # 2. Формируем словарь переименования в зависимости от наличия столбцов
        rename_map = {"tv_seriesLabel": "tv_series", "subjectLabel": "subject"}
        if "awardLabel" in df_clean.columns:
            rename_map["awardLabel"] = "award"

        df_clean = df_clean.rename(columns=rename_map)

        # 3. Приводим числовые столбцы к целочисленному типу
        for col in ["start_year", "end_year"]:
            if col in df_clean.columns:
                df_clean[col] = pd.to_numeric(
                    df_clean[col], errors="coerce"
                ).fillna(0).astype(int)

        print(f"✅ {name}: данные успешно очищены")
        status = "очищен"
    else:
        print(f"ℹ️  {name}: данные уже очищены ранее — повторная обработка пропущена")
        status = "уже чист"

    # Выводим структуру
    print(f"   Столбцы: {', '.join(df_clean.columns)}")
    print(f"   Строк: {len(df_clean)}\n")

    return df_clean, status

# Применяем очистку к обоим датасетам
print("🔧 Очистка датасетов:\n" + "="*70)

df_tv_one, status_one = clean_tv_dataset(df_tv_one, "Датасет 1 (с наградами)")
df_tv_two, status_two = clean_tv_dataset(df_tv_two, "Датасет 2 (с тематикой)")

print("="*70)

# Показываем примеры данных после очистки
print("\n📋 Пример данных после очистки:")
print("\n📺 Датасет 1 — первые 3 строки:")
display(df_tv_one.head(3))

print("\n📺 Датасет 2 — первые 3 строки:")
display(df_tv_two.head(3))

🔧 Очистка датасетов:
✅ Датасет 1 (с наградами): данные успешно очищены
   Столбцы: tv_series, start_year, end_year, award, subject
   Строк: 1224

✅ Датасет 2 (с тематикой): данные успешно очищены
   Столбцы: tv_series, start_year, end_year, subject
   Строк: 2679


📋 Пример данных после очистки:

📺 Датасет 1 — первые 3 строки:


,tv_series,start_year,end_year,award,subject
0,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучший телевизионный сериа...,NaN
1,Меня зовут Эрл,2005,2009,премия «Спутник» за лучшую женскую роль в теле...,NaN
2,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучшую женскую роль второг...,NaN



📺 Датасет 2 — первые 3 строки:


,tv_series,start_year,end_year,subject
0,Анна Франк,2001,2001,Вторая мировая война
1,Ходячие мертвецы,2010,2022,навык выживания
2,Государственная тайна (мини-сериал),2012,2012,политика


## 🔍 [3] Обзор данных: структура и сравнение двух датасетов

У нас есть **два связанных датасета** о телесериалах с разным фокусом:

| Датасет | Содержание | Особенности |
|---------|------------|-------------|
| `df_tv_one` | Сериалы с наградами | Есть столбцы `award` и `subject` — информация о премиях и номинациях |
| `df_tv_two` | Сериалы с тематикой | Только столбец `subject` — тематические категории без наград |

**В этом шаге мы:**
- посмотрим размер и структуру каждого датасета (`shape`, столбцы);
- выведем первые строки для понимания содержания;
- проанализируем распределение по годам выхода;
- сравним уникальные сериалы: какие есть в обоих датасетах, какие — только в одном;
- оценим разнообразие тем (`subject`) в каждом наборе.

Это поможет понять, как дополняют друг друга два источника и где возможны пробелы в данных.

In [4]:
def show_info(df, name, n=8):
    """Краткий обзор DataFrame с учётом возможных различий в структуре."""
    print(f"\n📊 {name}")
    print(f"Размер: {df.shape[0]} строк × {df.shape[1]} столбцов")
    print("Столбцы:", ", ".join(df.columns))

    # Фильтруем аномальные годы для статистики
    valid_years = df[df["start_year"] > 0]
    if len(valid_years) > 0:
        year_range = f"{valid_years['start_year'].min()}–{valid_years['end_year'].max()}"
        avg_duration = (valid_years["end_year"] - valid_years["start_year"]).mean()
        print(f"Годы выпуска: {year_range} (средняя длительность: {avg_duration:.1f} лет)")

    # Уникальные сериалы
    print(f"Уникальных сериалов: {df['tv_series'].nunique()}")

    # Темы и награды (если есть)
    if "subject" in df.columns:
        non_empty_subjects = df["subject"].notna().sum()
        print(f"Тематических номинаций: {non_empty_subjects} (уникальных тем: {df['subject'].nunique()})")
    if "award" in df.columns:
        non_empty_awards = df["award"].notna().sum()
        print(f"Наград/номинаций: {non_empty_awards} (уникальных наград: {df['award'].nunique()})")

    print("\nПервые строки:")
    display(df.head(n))

# 🔍 Шаг 3. Обзор и сравнение двух датасетов

print("="*70)
print("📺 ОБЗОР ДАТАСЕТОВ")
print("="*70)

show_info(df_tv_one, "Датасет 1: Сериалы с наградами (df_tv_one)")
show_info(df_tv_two, "Датасет 2: Сериалы с тематикой (df_tv_two)")

print("\n" + "="*70)
print("🔄 СРАВНЕНИЕ ДАТАСЕТОВ: пересечения и различия")
print("="*70)

# Уникальные сериалы в каждом датасете
series_one = set(df_tv_one["tv_series"].unique())
series_two = set(df_tv_two["tv_series"].unique())

intersection = series_one & series_two
only_one = series_one - series_two
only_two = series_two - series_one

print(f"\n📈 Общая статистика:")
print(f"  Всего сериалов в датасете 1: {len(series_one)}")
print(f"  Всего сериалов в датасете 2: {len(series_two)}")
print(f"  Сериалов, присутствующих в ОБОИХ датасетах: {len(intersection)}")
print(f"  Сериалов ТОЛЬКО в датасете 1 (с наградами): {len(only_one)}")
print(f"  Сериалов ТОЛЬКО в датасете 2 (без наград): {len(only_two)}")

# Примеры сериалов только во втором датасете (интересно для анализа)
if len(only_two) > 0:
    print(f"\n💡 Примеры сериалов, которые есть ТОЛЬКО во втором датасете (тематика без наград):")
    example_series = sorted(list(only_two))[:5]
    for i, s in enumerate(example_series, 1):
        # Находим годы для этого сериала
        years = df_tv_two[df_tv_two["tv_series"] == s][["start_year", "end_year"]].iloc[0]
        year_str = f"{years['start_year']}" if years['end_year'] == 0 else f"{years['start_year']}-{years['end_year']}"
        print(f"   {i}. {s} ({year_str})")

print("\n" + "="*70)
print("💬 ТЕМАТИЧЕСКОЕ РАЗНООБРАЗИЕ: Топ-10 тем в объединённых данных")
print("="*70)

# Объединяем темы из обоих датасетов для полной картины
all_subjects = pd.concat([
    df_tv_one["subject"].dropna(),
    df_tv_two["subject"].dropna()
])

top_subjects = all_subjects.value_counts().head(10)
print(f"\nВсего уникальных тем в объединённых данных: {all_subjects.nunique()}")
print(f"\nТоп-10 самых частых тем:")
for rank, (subject, count) in enumerate(top_subjects.items(), 1):
    pct = count / len(all_subjects) * 100
    print(f"{rank:2d}. {subject:.<45} {count:4d} ({pct:4.1f}%)")

📺 ОБЗОР ДАТАСЕТОВ

📊 Датасет 1: Сериалы с наградами (df_tv_one)
Размер: 1224 строк × 5 столбцов
Столбцы: tv_series, start_year, end_year, award, subject
Годы выпуска: 1937–2024 (средняя длительность: -207.5 лет)
Уникальных сериалов: 294
Тематических номинаций: 447 (уникальных тем: 73)
Наград/номинаций: 1224 (уникальных наград: 277)

Первые строки:


,tv_series,start_year,end_year,award,subject
0,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучший телевизионный сериа...,NaN
1,Меня зовут Эрл,2005,2009,премия «Спутник» за лучшую женскую роль в теле...,NaN
2,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучшую женскую роль второг...,NaN
3,Меня зовут Эрл,2005,2009,Премия Гильдии киноактёров США за лучшую женск...,NaN
4,Меня зовут Эрл,2005,2009,Премия Гильдии режиссёров Америки за лучшую ре...,NaN
5,Меня зовут Эрл,2005,2009,прайм-тайм премия «Эмми» в категории «Лучший п...,NaN
6,Меня зовут Эрл,2005,2009,Премия Ассоциации телевизионных критиков за вы...,NaN
7,Меня зовут Эрл,2005,2009,Премия «Золотой глобус» за лучший телевизионны...,NaN



📊 Датасет 2: Сериалы с тематикой (df_tv_two)
Размер: 2679 строк × 4 столбцов
Столбцы: tv_series, start_year, end_year, subject
Годы выпуска: 1948–2026 (средняя длительность: -621.6 лет)
Уникальных сериалов: 1878
Тематических номинаций: 2679 (уникальных тем: 1713)

Первые строки:


,tv_series,start_year,end_year,subject
0,Анна Франк,2001,2001,Вторая мировая война
1,Ходячие мертвецы,2010,2022,навык выживания
2,Государственная тайна (мини-сериал),2012,2012,политика
3,Вице-президент,2012,2019,политика
4,Спин-Сити,1996,2002,политика
5,Штамм «Андромеда»,2008,2008,внеземной микроорганизм
6,Dae Jo Yeong,2006,2007,война
7,Кости,2005,2017,Федеральное бюро расследований



🔄 СРАВНЕНИЕ ДАТАСЕТОВ: пересечения и различия

📈 Общая статистика:
  Всего сериалов в датасете 1: 294
  Всего сериалов в датасете 2: 1878
  Сериалов, присутствующих в ОБОИХ датасетах: 49
  Сериалов ТОЛЬКО в датасете 1 (с наградами): 245
  Сериалов ТОЛЬКО в датасете 2 (без наград): 1829

💡 Примеры сериалов, которые есть ТОЛЬКО во втором датасете (тематика без наград):
   1. 1,000 Places to See Before You Die (2007)
   2. 10 Years Younger (2004-2009)
   3. 100 Great Paintings (0)
   4. 101 Cars You Must Drive (2008-2008)
   5. 112 – Le notti del Radiomobile (2024)

💬 ТЕМАТИЧЕСКОЕ РАЗНООБРАЗИЕ: Топ-10 тем в объединённых данных

Всего уникальных тем в объединённых данных: 1713

Топ-10 самых частых тем:
 1. неблагополучная семья........................  121 ( 3.9%)
 2. Вторая мировая война.........................   74 ( 2.4%)
 3. политика.....................................   69 ( 2.2%)
 4. Департамент полиции города Нью-Йорка.........   65 ( 2.1%)
 5. ремонт жилища......................

## 🔬 [4] Глубокая аналитика: наградный дефицит и долгожители без признания

Наши данные раскрывают **две культурные аномалии телевизионной индустрии**:

### ⚖️ Наградный дефицит по темам
Не все темы получают равное признание. Мы посчитаем для каждой темы:
- **Общее число сериалов** с этой темой (из обоих датасетов);
- **Число сериалов с наградами**;
- **Коэффициент признания** = (сериалы с наградами) / (все сериалы с темой).

Низкий коэффициент (< 0.1) означает **системное игнорирование** темы премиальными системами — даже при большом количестве сериалов на эту тему.

### 🕰️ Долгожители без наград
Сериалы, которые шли **10+ лет**, но не получили ни одной премии. Это парадокс: долгий успех у зрителей ≠ признание критиков. Анализ выявит:
- Сериалы с максимальной продолжительностью без наград;
- Возможные причины: нишевая аудитория, документальный формат, региональное производство.

### 🤝 Особый фокус: дружба на экране
Завершим анализ поиском сериалов с темой **«дружба»** — универсальной ценности, которую индустрия может недооценивать (судя по отсутствию в Топ-10 тем).

In [5]:
# 🔬 Шаг 4. Наградный дефицит и долгожители без признания

import pandas as pd

print("="*70)
print("⚖️  АНАЛИЗ 1: НАГРАДНЫЙ ДЕФИЦИТ ПО ТЕМАМ")
print("="*70)

# Подготовка: объединяем темы из обоих датасетов с пометкой источника
df_one_with_source = df_tv_one[["tv_series", "subject"]].copy()
df_one_with_source["has_award"] = True
df_one_with_source = df_one_with_source[df_one_with_source["subject"].notna() & (df_one_with_source["subject"] != "")]

df_two_with_source = df_tv_two[["tv_series", "subject"]].copy()
df_two_with_source["has_award"] = False
df_two_with_source = df_two_with_source[df_two_with_source["subject"].notna() & (df_two_with_source["subject"] != "")]

# Объединяем для анализа
df_themes = pd.concat([df_one_with_source, df_two_with_source], ignore_index=True)

# Статистика по каждой теме
theme_stats = df_themes.groupby("subject").agg(
    total_series=("tv_series", "count"),
    awarded_series=("has_award", "sum"),
).reset_index()

# Коэффициент признания (с защитой от деления на ноль)
theme_stats["recognition_ratio"] = (
    theme_stats["awarded_series"] / theme_stats["total_series"]
).round(3)

# Сортируем: сначала темы с самым низким признанием
theme_stats = theme_stats.sort_values("recognition_ratio").reset_index(drop=True)

# Фильтруем значимые темы (минимум 5 сериалов для статистической значимости)
theme_stats_significant = theme_stats[theme_stats["total_series"] >= 5].copy()

print(f"Всего тем с ≥5 сериалами: {len(theme_stats_significant)} из {len(theme_stats)}")
print(f"Тем с нулевым признанием (0 наград): {(theme_stats_significant['recognition_ratio'] == 0).sum()}\n")

print("📉 Топ-15 тем с НАИБОЛЬШИМ наградным дефицитом (≥5 сериалов):")
print("-" * 70)
print(f"{'Ранг':<6}{'Тема':<40}{'Сериалов':<10}{'Наград':<10}{'Коэф.':<8}")
print("-" * 70)

for idx, row in theme_stats_significant.head(15).iterrows():
    rank = idx + 1
    theme = row["subject"][:38] + "..." if len(row["subject"]) > 38 else row["subject"]
    print(f"{rank:<6}{theme:<40}{int(row['total_series']):<10}{int(row['awarded_series']):<10}{row['recognition_ratio']:<8.1%}")

print("\n💡 Инсайт: темы с коэффициентом < 10% системно игнорируются премиями,")
print("   даже при наличии десятков сериалов на эту тему.")

print("\n" + "="*70)
print("🕰️  АНАЛИЗ 2: ДОЛГОЖИТЕЛИ БЕЗ НАГРАД (≥10 лет в эфире)")
print("="*70)

# Подготовка данных с фильтрацией аномальных годов
def calculate_duration(df):
    """Безопасный расчёт продолжительности с фильтрацией аномалий."""
    df_clean = df[
        (df["start_year"] >= 1900) &
        (df["start_year"] <= 2024) &
        (df["end_year"] >= df["start_year"]) &
        (df["end_year"] <= 2024)
    ].copy()
    df_clean["duration"] = df_clean["end_year"] - df_clean["start_year"]
    return df_clean

df_two_clean = calculate_duration(df_tv_two)
df_one_clean = calculate_duration(df_tv_one)

# Сериалы-долгожители без наград: есть в датасете 2, но нет в датасете 1
longrunners_no_awards = df_two_clean[
    (df_two_clean["duration"] >= 10) &
    (~df_two_clean["tv_series"].isin(df_one_clean["tv_series"]))
].copy()

longrunners_no_awards = longrunners_no_awards.sort_values("duration", ascending=False).reset_index(drop=True)

print(f"Найдено долгожителей без наград (≥10 лет): {len(longrunners_no_awards)}\n")

if len(longrunners_no_awards) > 0:
    print("📺 Топ-10 самых продолжительных сериалов без наград:")
    print("-" * 70)
    print(f"{'Ранг':<6}{'Сериал':<35}{'Годы':<15}{'Длит.':<8}{'Темы'}")
    print("-" * 70)

    for idx, row in longrunners_no_awards.head(10).iterrows():
        rank = idx + 1
        series = row["tv_series"][:33] + "..." if len(row["tv_series"]) > 33 else row["tv_series"]
        years = f"{int(row['start_year'])}-{int(row['end_year'])}"
        duration = int(row["duration"])
        subject = row["subject"][:25] + "..." if len(str(row["subject"])) > 25 else str(row["subject"])
        print(f"{rank:<6}{series:<35}{years:<15}{duration:<8}{subject}")

    print("\n💡 Гипотезы почему нет наград:")
    print("   • Документальные/реалити-форматы реже номинируются")
    print("   • Региональные сериалы (не англоязычные рынки)")
    print("   • Нишевая аудитория (детские, образовательные шоу)")
else:
    print("✅ Все долгожители (≥10 лет) получили хотя бы одну награду!")

print("\n" + "="*70)
print("🤝 СЕРИАЛЫ С ТЕМОЙ «ДРУЖБА»")
print("="*70)

# Поиск по обоим датасетам (регистронезависимо, с учётом опечаток)
def find_friendship(df, has_awards=False):
    """Поиск сериалов с темой дружбы."""
    # Ищем разные варианты написания
    mask = (
        df["subject"].str.contains("дружб", case=False, na=False) |  # ловит "дружба", "дружбе" и т.д.
        df["subject"].str.contains("friendship", case=False, na=False) |
        df["subject"].str.contains("bromance", case=False, na=False) |
        df["subject"].str.contains("camaraderie", case=False, na=False)
    )
    if has_awards:
        return df[mask][["tv_series", "start_year", "end_year", "award", "subject"]].drop_duplicates()
    else:
        return df[mask][["tv_series", "start_year", "end_year", "subject"]].drop_duplicates()

df_friendship_awarded = find_friendship(df_tv_one, has_awards=True)
df_friendship_no_awards = find_friendship(df_tv_two, has_awards=False)

# Исключаем дубликаты из второго датасета, если сериал уже есть в первом
if len(df_friendship_awarded) > 0:
    awarded_series = set(df_friendship_awarded["tv_series"])
    df_friendship_no_awards = df_friendship_no_awards[~df_friendship_no_awards["tv_series"].isin(awarded_series)]

total_friendship = len(df_friendship_awarded) + len(df_friendship_no_awards)

if total_friendship > 0:
    print(f"Найдено {total_friendship} записей, связанных с дружбой:\n")

    if len(df_friendship_awarded) > 0:
        print(f"🏆 Сериалы с наградами ({len(df_friendship_awarded)}):")
        for idx, row in df_friendship_awarded.iterrows():
            years = f"{int(row['start_year'])}-{int(row['end_year'])}" if row['end_year'] > row['start_year'] else str(int(row['start_year']))
            print(f"   • {row['tv_series']} ({years})")
            print(f"     Награда: {row['award']}")
            print(f"     Тема:    {row['subject']}\n")

    if len(df_friendship_no_awards) > 0:
        print(f"📺 Сериалы без наград ({len(df_friendship_no_awards)}):")
        for idx, row in df_friendship_no_awards.iterrows():
            years = f"{int(row['start_year'])}-{int(row['end_year'])}" if row['end_year'] > row['start_year'] else str(int(row['start_year']))
            print(f"   • {row['tv_series']} ({years}) — {row['subject']}")
else:
    print("❌ Не найдено записей, содержащих «дружб» или «friendship».")
    print("\n🔎 Рекомендации для углублённого поиска:")
    print("   1. Расширенный поиск по синонимам: «отношения», «команда», «приятели»")
    print("   2. Анализ сюжетных описаний (не только subjectLabel)")
    print("   3. Проверка английских Викиданных: «buddy», «sidekick», «allies»")
    print("\n💡 Культурный вывод: тема дружбы может быть «растворена» в других категориях")
    print("   («семья», «коллеги», «партнёры»), а не выделена как отдельная номинация.")

⚖️  АНАЛИЗ 1: НАГРАДНЫЙ ДЕФИЦИТ ПО ТЕМАМ
Всего тем с ≥5 сериалами: 63 из 1713
Тем с нулевым признанием (0 наград): 26

📉 Топ-15 тем с НАИБОЛЬШИМ наградным дефицитом (≥5 сериалов):
----------------------------------------------------------------------
Ранг  Тема                                    Сериалов  Наград    Коэф.   
----------------------------------------------------------------------
91    гомицид                                 5         0         0.0%    
94    глобальное потепление                   5         0         0.0%    
191   история                                 6         0         0.0%    
222   Холодная война                          5         0         0.0%    
253   Центральное разведывательное управлени...5         0         0.0%    
260   Холокост                                7         0         0.0%    
370   авиация                                 5         0         0.0%    
395   антиквариат                             23        0         0.0%    
40

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨